In [1]:
import os.path
import glob
import math
import pandas as pd
import numpy as np
import datascience

In [2]:
# Function Definitions

# MODIS State QA

def isLand(bits):
    return (bits & 0x0038) == 0x0008
    
def isnotCloudMOD(bits):
    return (bits & 0x0003) == 0
    
def isnotShadowMOD(bits):
    return (bits & 0x0004) == 0
    
def isAeroLow(bits):
    return (bits & 0x00c0) == 0x0040

def isnotCirrus(bits):
    return (bits & 0x0300) == 0

def isnotCloudINT(bits):
    return (bits & 0x0400) == 0

def isnotAdjCloud(bits):
    return (bits & 0x2000) == 0

def isnotSnowMOD(bits):
    return (bits & 0x8000) == 0

def isnotSnowINT(bits):
    return (bits & 0x1000) == 0

#make a new coulmn in CSV with the 1 or 0, and that will tell me whether to include that row/pixel 
def stateQAtest(QA):
    return [(isLand(QA))&
            (isnotCloudMOD(QA))]

In [3]:
# CONSTANTS / CONFIGURABLE VALUES

# Directory to scan for CSV files and save output TXT
root = "/Volumes/SD/radCats"

In [4]:
# Find all CSV files including inside sub-directories csvFiles = glob.glob(os.path.join(root, "**/*.czs"))
csvFiles = glob.glob(os.path.join(root, "**", "*.csv"), recursive=True)
print("Files: ", len(csvFiles))

Files:  4469


In [5]:
# Loop over every CSV file found
for csvFile in csvFiles:
    import csv 
    with open(csvFile, "r") as f: 
        modisFile = list(csv.reader(f, delimiter = ','))
    #Creates an array of the stateQA values
    modisArray = np.array(modisFile[0:], dtype=None)
    qaArray = modisArray[:,3].astype(np.uint16, copy=False)
    dataStateQA = stateQAtest(qaArray)
#re-shapes the numpy array of QA test results to be in a column, not a list like a row
    reshapeData = np.reshape(dataStateQA, (24,1))
    csv_data = pd.DataFrame(reshapeData)
 #Creates a pandas dataframe from the MODIS csv   
    pandaModisFile = pd.DataFrame(modisFile)
    #concatenates the State QA test results and the original data into one dataframe
    appendQA = pd.concat([pandaModisFile,csv_data],axis=1, ignore_index=True)
    #print(appendQA)   
    #np.savetxt(csvFile, reshapeData, delimiter=",")   
#Uncomment to write to file
    appendQA.to_csv(csvFile, header=None, index=None)
   
    #break # Remove this to make it run for all files, not just the first

In [6]:
root = "/Volumes/SD/radCats"

In [7]:
csvFilesQA = glob.glob(os.path.join(root, "**", "*.csv"), recursive=True)
print("Files: ", len(csvFilesQA))

Files:  4469


In [8]:
for csvFileQA in csvFilesQA:
    import csv
    with open(csvFileQA, "r") as f: 
        modisFileQA = list(csv.reader(f, delimiter = ','))
  #creates a pandas dataframe of the modis CSV+QA results  
    pandaPixels = pd.DataFrame(modisFileQA)
#indexes the pd dataframe to exclude non-numeric series, and changes the datatype to numeric
    studySitePixels = pandaPixels.loc[[9,10,11,14,15,16,19,20,21],3:17].reset_index(drop=True).apply(pd.to_numeric)
    #Isolates the state QA result series
    studySitePixelsQAresult = pandaPixels.loc[[9,10,11,14,15,16,19,20,21],18:]
    #studySitePixelsAll = pandaPixels.loc[[9,10,11,14,15,16,19,20,21], : ]
    
#runs summary statistics on the desired pixels in the study site
    summaryStats = studySitePixels.describe().loc[['mean','min','max','std']]
  
    ##concatenates the summary stat results with the original data
    appSummaryStats = pd.concat([pandaPixels, summaryStats]).reset_index(drop=True)
    
    #makes sure that the file name coulmn is filled out all the way
    headerCol = appSummaryStats.loc[:,0]
    fileName = pandaPixels.loc[0,0]
    filled = headerCol.fillna(fileName)
    
    bodyValues = appSummaryStats.loc[:,1:]
    completeStats = pd.concat([filled, bodyValues], axis=1)
    
    #completeStats.to_csv(csvFileQA, header=False)
    #print(completeStats)
    
    #qaSummary = studySitePixelsQAresult.all()
    
    #if qaSummary.all() == 1:
    completeStats.to_csv(csvFileQA, header=None, index=None)
   # else: 
     #   pandaPixels.to_csv(csvFileQA, header=None, index=None)
      #  print(studySitePixelsQAresult.all()) 
 


In [18]:
root = "/Volumes/SD/radCalNet/test"

In [19]:
csvFilesStat = glob.glob(os.path.join(root, "**", "*.csv"), recursive=True)
print("Files: ", len(csvFilesStat))

Files:  2


In [44]:
for csvFileStat in csvFilesStat:
    import csv
    with open(csvFileStat, "r") as f: 
        modisFileStat = list(csv.reader(f, delimiter = ','))
  #creates a pandas dataframe of the modis CSV+QA results+summaryStats  
    pandaPixels = pd.DataFrame(modisFileStat)
    meanPixels = pandaPixels.loc[[24],0:]
    minPixels = pandaPixels.loc[[25],0:]
    maxPixels = pandaPixels.loc[[26],0:]
    
    mean = pd.DataFrame(data=None)
    
    meanCSV = pd.concat([mean,meanPixels])
    
    

In [45]:
meanCSV

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
24,MYD09GA.A2018059.h08v05.006.2018061030252,,,9302.222222222223,4464.222222222223,7507.111111111111,4674.222222222223,-17002.11111111111,3002.5555555555557,3445.3333333333335,1579.6666666666667,2569.0,3429.222222222222,-3407.777777777778,3356.222222222222,0.06866219633333334,0.025556973111111117,0.053590220888888894,


In [37]:
minPixels

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
25,MYD09GA.A2018059.h08v05.006.2018061030252,,,200.0,4462.0,7502.0,4674.0,-17005.0,2902.0,3343.0,1506.0,2474.0,3315.0,-28672.0,3261.0,0.065989848,0.024355116,0.050916497000000005,


In [38]:
maxPixels

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
26,MYD09GA.A2018059.h08v05.006.2018061030252,,,16584.0,4472.0,7514.0,4675.0,-17001.0,3150.0,3609.0,1685.0,2705.0,3554.0,3863.0,3460.0,0.071726053,0.026827128999999998,0.05617136,


In [244]:
print(qaSummary.all())

True


In [69]:
fillColoumnName = appSummaryStats.loc[24:,0] = appSummaryStats.loc[0,0]




              3           4             5            6             7   \
mean  200.444444  176.777778  11918.333333  6327.555556  16210.888889   
min   200.000000  166.000000  11748.000000  6327.000000  16210.000000   
max   202.000000  186.000000  12113.000000  6329.000000  16212.000000   
std     0.881917    5.651942    132.893755     0.726483      0.600925   

               8            9            10           11           12  \
mean  3459.000000  3814.000000  1771.000000  2972.888889  3768.888889   
min   3311.000000  3665.000000  1667.000000  2838.000000  3293.000000   
max   3564.000000  3921.000000  1913.000000  3112.000000  4071.000000   
std     84.886689    82.073138    85.562842    91.506071   302.557203   

               13           14        15        16        17   0   
mean  3983.888889  3304.222222  0.048842  0.018562  0.040140  NaN  
min   3781.000000  2957.000000  0.045713  0.017314  0.037836  NaN  
max   4204.000000  3493.000000  0.050976  0.019508  0.041530  Na

In [151]:
print(studySitePixels)

    3    4      5     6      7     8     9     10    11    12    13    14  \
0  200  186  12014  6329  16210  3547  3899  1913  3112  3455  4204  3102   
1  202  178  12113  6328  16211  3311  3665  1702  2838  3293  4003  2957   
2  202  178  12113  6328  16211  3408  3770  1744  2900  4057  3781  3484   
3  200  175  11836  6327  16211  3379  3742  1667  2883  3877  3850  3344   
4  200  175  11836  6327  16211  3446  3800  1768  2974  3540  4035  3205   
5  200  166  11933  6327  16212  3564  3921  1853  3060  4055  3989  3493   
6  200  183  11748  6328  16210  3438  3797  1667  2929  3994  3912  3440   
7  200  175  11836  6327  16211  3510  3866  1789  3028  3578  4120  3259   
8  200  175  11836  6327  16211  3528  3866  1836  3032  4071  3961  3454   

            15           16           17  
0  0.047273704  0.017767548   0.03926503  
1  0.050745413  0.019116122  0.040950609  
2  0.050431875  0.019134001  0.041231571  
3  0.050975987  0.019508368   0.04153014  
4   0.04885454

In [ ]:
def stateQAtest(QA):
    return [(isLand(QA))&
            (isnotCloudMOD(QA))&
            (isnotShadowMOD(QA))&
            isAeroLow(QA)&
            isnotCirrus(QA)&
            isnotSnowMOD(QA)&
            isnotCloudINT(QA)&
            isnotSnowMOD(QA)&
            isnotSnowINT(QA)]

In [111]:
BBB = studySitePixels.mean()
print(BBB)

3     2.224469e+25
4     2.068646e+25
5     1.334902e+43
6     7.032925e+34
7     1.801129e+43
8     3.941479e+34
9     4.332629e+34
10    2.125745e+34
11    3.458093e+34
12    3.839255e+34
13    4.671556e+34
14    3.446995e+34
dtype: float64


In [54]:
# CONSTANTS / CONFIGURABLE VALUES

# Directory to scan for CSV files and save output TXT
root = "/Volumes/SD/radCalNet/test"

In [55]:
# Find all CSV files including inside sub-directories csvFiles = glob.glob(os.path.join(root, "**/*.czs"))
csvFiles = glob.glob(os.path.join(root, "**", "*.csv"), recursive=True)
print("Files: ", len(csvFiles))

Files:  1


In [56]:
# Loop over every CSV file found
for csvFile in csvFiles:
    import csv 
    with open(csvFile, "r") as f: 
        modisFile = list(csv.reader(f, delimiter = ','))
    #Creates an array of the stateQA values
    modisArray = np.array(modisFile[0:], dtype=None)
    qaArray = modisArray[:,3].astype(np.uint16, copy=False)
    dataStateQA = stateQAtest(qaArray)
#re-shapes the numpy array of QA test results to be in a column, not a list like a row
    reshapeData = np.reshape(dataStateQA, (24,1))
    csv_data = pd.DataFrame(reshapeData)
 #Creates a pandas dataframe from the MODIS csv   
    pandaModisFile = pd.DataFrame(modisFile)
    
    appendQA = pd.concat([pandaModisFile,csv_data],axis=1, ignore_index=True)
    print(appendQA)   
    #np.savetxt(csvFile, reshapeData, delimiter=",")   
#Uncomment to write to file
    appendQA.to_csv(csvFile, header=None, index=None)
   
    #break # Remove this to make it run for all files, not just the first

                                           0  1  2     3    4      5     6   \
0   MOD09GA.A2018004.h08v05.006.2018006030625  0  0  8394  195  11923  6329   
1   MOD09GA.A2018004.h08v05.006.2018006030625  0  1  8394  195  11923  6329   
2   MOD09GA.A2018004.h08v05.006.2018006030625  0  2   200  186  12014  6329   
3   MOD09GA.A2018004.h08v05.006.2018006030625  0  3   200  186  12014  6329   
4   MOD09GA.A2018004.h08v05.006.2018006030625  0  4   202  178  12113  6328   
5   MOD09GA.A2018004.h08v05.006.2018006030625  0  5   202  178  12113  6328   
6   MOD09GA.A2018004.h08v05.006.2018006030625  1  0  8394  195  11923  6329   
7   MOD09GA.A2018004.h08v05.006.2018006030625  1  1  8394  195  11923  6329   
8   MOD09GA.A2018004.h08v05.006.2018006030625  1  2   200  186  12014  6329   
9   MOD09GA.A2018004.h08v05.006.2018006030625  1  3   200  186  12014  6329   
10  MOD09GA.A2018004.h08v05.006.2018006030625  1  4   202  178  12113  6328   
11  MOD09GA.A2018004.h08v05.006.2018006030625  1  5 

In [81]:
import csv 
with open(csvFiles, "r") as f: 
    modis = list(csv.reader(f, delimiter = ','))
MOD = np.array(modis[0:], dtype=None)
qaqa = MOD[:,3].astype(np.uint16, copy=False)


In [51]:
qaqa = MOD[:,3]

In [78]:
print(qaqa)

[8393 8393 8393 8393 8393 8393 8393 8393 8393 8393 8393 8393 8393 8393
 8393 8393 1033 1033 8393 8393 8393 8393 1033 1033]


In [76]:
print(stateQAtest(qaqa))

[array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False])]


[array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False])]


In [27]:
# Function Definitions

# MODIS State QA

def isLand(bits):
    return (bits & 0x0038) == 0x0008
    
def isnotCloudMOD(bits):
    return (bits & 0x0003) == 0
    
def isnotShadowMOD(bits):
    return (bits & 0x0004) == 0
    
def isAeroLow(bits):
    return (bits & 0x00c0) == 0x0040

def isnotCirrus(bits):
    return (bits & 0x0300) == 0

def isnotCloudINT(bits):
    return (bits & 0x0400) == 0

def isnotAdjCloud(bits):
    return (bits & 0x2000) == 0

def isnotSnowMOD(bits):
    return (bits & 0x8000) == 0

def isnotSnowINT(bits):
    return (bits & 0x1000) == 0

#make a new coulmn in CSV with the 1 or 0, and that will tell me whether to include that row/pixel 
def stateQAtest(QA):
    return [(isLand(QA))&
            (isnotCloudMOD(QA))&
            (isnotShadowMOD(QA))&
            isAeroLow(QA)&
            isnotCirrus(QA)&
            isnotSnowMOD(QA)&
            isnotCloudINT(QA)&
            isnotSnowMOD(QA)&
            isnotSnowINT(QA)]

In [11]:
# CONSTANTS / CONFIGURABLE VALUES

# Directory to scan for CSV files and save output TXT
root = "/Volumes/SD/radCalNet/2018MYD09GA"

In [13]:
# Find all CSV files including inside sub-directories csvFiles = glob.glob(os.path.join(root, "**/*.czs"))
csvFiles = glob.glob(os.path.join(root, "**", "*.csv"), recursive=True)
print("Files: ", len(csvFiles))

Files:  59


In [69]:
# Loop over every CSV file found
for csvFile in csvFiles:
    csv_data = pandas.read_csv(csvFile, header=None)
    #Creates an array of the stateQA values
    stateQA = csv_data.loc[:,[3]]
    stateQATestResult = stateQAtest(stateQA)
    if all(stateQATestResult) is True
        return stateQA.mean(axis=0)
    #Append column to csv_data here
    
    
    
    #Uncomment to write to file
    #csv_data.to_csv(csvFile)

    break # Remove this to make it run for all files, not just the first

SyntaxError: invalid syntax (<ipython-input-69-f49315aea5b8>, line 7)

In [67]:
# Loop over every CSV file found
for csvFile in csvFiles:
    csv_data = pandas.read_csv(csvFile, header=None)
    #Creates an array of the stateQA values
    stateQA = csv_data.loc[:,[3]]
    stateQATestResult = stateQAtest(stateQA)
    
    #Append column to csv_data here
    
    csv_data.to_

0            3
0   False
1   False
2   False
3   Fa...
dtype: object


ValueError: Other Series must have a name

In [22]:
test = pandas.read_csv("/Users/Char/Desktop/MOD09GA.A2012001.h08v05.006.2015229234041.csv", header=None)

In [23]:
print(test)

                                           0   1   2     3     4     5     6   \
0   MOD09GA.A2012001.h08v05.006.2015229234041   0   0  8393  4804  9638  6571   
1   MOD09GA.A2012001.h08v05.006.2015229234041   0   1  8393  4804  9638  6571   
2   MOD09GA.A2012001.h08v05.006.2015229234041   0   2  8393  4803  9553  6571   
3   MOD09GA.A2012001.h08v05.006.2015229234041   0   3  8393  4803  9553  6571   
4   MOD09GA.A2012001.h08v05.006.2015229234041   0   4  8393  4793  9554  6570   
5   MOD09GA.A2012001.h08v05.006.2015229234041   0   5  8393  4793  9554  6570   
6   MOD09GA.A2012001.h08v05.006.2015229234041   1   0  8393  4804  9638  6571   
7   MOD09GA.A2012001.h08v05.006.2015229234041   1   1  8393  4804  9638  6571   
8   MOD09GA.A2012001.h08v05.006.2015229234041   1   2  8393  4803  9553  6571   
9   MOD09GA.A2012001.h08v05.006.2015229234041   1   3  8393  4803  9553  6571   
10  MOD09GA.A2012001.h08v05.006.2015229234041   1   4  8393  4793  9554  6570   
11  MOD09GA.A2012001.h08v05.

In [24]:
stateQA = test.loc[:,[3]]

In [25]:
print(stateQA)

       3
0   8393
1   8393
2   8393
3   8393
4   8393
5   8393
6   8393
7   8393
8   8393
9   8393
10  8393
11  8393
12  8393
13  8393
14  8393
15  8393
16  1033
17  1033
18  8393
19  8393
20  8393
21  8393
22  1033
23  1033


In [81]:
testResult = (stateQAtest(stateQA))

In [88]:
print(testResult)

[        3
0   False
1   False
2   False
3   False
4   False
5   False
6   False
7   False
8   False
9   False
10  False
11  False
12  False
13  False
14  False
15  False
16  False
17  False
18  False
19  False
20  False
21  False
22  False
23  False]


In [95]:
 pandaNP = np.array(modisFileQA)
    
    indexNP = pandaNP[:24, 3:18]
    
    topRow = indexNP[9:12, :]
    middleRow= indexNP[14:17, :]
    bottomRow= indexNP[18:21, :]
    
    catRows = np.concatenate([topRow, middleRow, bottomRow])
    
    print(np.mean(topRow))
    
    print(catRows)

In [78]:
# def stateQAtest(QA):
#     bit_results = [isLand(QA),
#     isnotCloudMOD(QA),
#     isnotShadowMOD(QA),
#     isAeroLow(QA),
#     isnotCirrus(QA),
#     isnotCloudINT(QA),
#     isnotSnowMOD(QA),
#     isnotSnowINT(QA)]
    
#     row_results = []
#     for row in QA[3]:
#         bit_results = [isLand(row),
#             isnotCloudMOD(row),
#             isnotShadowMOD(row),
#             isAeroLow(row),
#             isnotCirrus(row),
#             isnotCloudINT(row),
#             isnotSnowMOD(row),
#             isnotSnowINT(row)]
#         (bit_results)
#         row_results.append(all(bit_results))
    
#     return row_results
#     print(bit_results)
# NF=9
# flag = [0]*9
    
# for i in range(1, NF):
#     flag[0] = flag[0] * flag[i]
 
# if flag[0] == 1:
#     pass 
# else:
#     pass 
    

In [ ]:
# #if it is true put 1 if false put 0

# # /* MODIS State QA */
# #define NF 9
# NF = 9
# #define isnotDeepOcean(bits)    ((0x0038 & (bits)) != 0x0038)
# #define isnotModOcean(bits)     ((0x0038 & (bits)) != 0x0030)
# #define isnotShallowOcean(bits) ((0x0038 & (bits)) != 0)

# # flag = 0;
# flag = [0] * 9

# # yes #define isLand(bits)            ((0x0038 & (bits)) == 0x0008)     flag[0]=1
# def isLand(bits):
#     flag[0] = 1
#     return (bits & 0x0038) == 0x0008
# # yes #define isnotCloudMOD(bits)     ((0x0003 & (bits)) == 0)        flag[1]=1
# def isnotCloudMOD(bits):
#     flag[1] = 1
#     return (bits & 0x0003) == 0
# # yes #define isnotShadowMOD(bits)    ((0x0004 & (bits)) == 0)     flag[2]=1
# def isnotShadowMOD(bits):
#     flag[2] = 1
#     return (bits & 0x0004) == 0
# # #define isAeroClm(bits)         ((0x00c0 & (bits)) == 0)
# # yes #define isAeroLow(bits)         ((0x00c0 & (bits)) == 0x0040)
# def isAeroLow(bits):
#     return (bits & 0x00c0) == 0x0040
# # #define isAeroMed(bits)         ((0x00c0 & (bits)) == 0x0080)
# # #define isnotAeroHigh(bits)     ((0x00c0 & (bits)) != 0x00c0)
# # yes #define isnotCirrus(bits)       ((0x0300 & (bits)) == 0)
# def isnotCirrus(bits):
#     return (bits & 0x0300) == 0
# # yes #define isnotCloudINT(bits)     ((0x0400 & (bits)) == 0)
# def isnotCloudINT(bits):
#     return (bits & 0x0400) == 0
# # yes #define isnotAdjCloud(bits)     ((0x2000 & (bits)) == 0)
# def isnotAdjCloud(bits):
#     return (bits & 0x2000) == 0
# # yes #define isnotSnowMOD(bits)      ((0x8000 & (bits)) == 0)
# def isnotSnowMOD(bits):
#     return (bits & 0x8000) == 0
# # yes #define isnotSnowINT(bits)      ((0x1000 & (bits)) == 0)
# def isnotSnowINT(bits):
#     return (bits & 0x1000) == 0


# # multiples all nine flags and if any of flags is 0 then the result is 0, 
# #and zero means do not keep the pixel 
# # for(i=1; i<; i++){
# # 	flag[0] = flag[0] * flag[i];  // flag[0] *= flag[i];
# # }
# for i in range(1, NF):
#     flag[0] = flag[0] * flag[i]

# # if(flag[0] == 1){ 
# # 	/* we use this observation set the number to 1 include it in mean computation */
# # } else {
# #   /* assign 0 to the row */
# # }

# if flag[0] == 1:
#     pass
# else:
#     pass

#make a new coulmn in CSV with the 1 or 0, and that will tell me whether to include that row/pixel 